# Assignment 4: Self-Attention for Vision

For this assignment, we're going to implement self-attention blocks in a convolutional neural network for CIFAR-10 Classification.

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./data/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./data/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./data/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


## Flatten Function

In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Check Accuracy Function


In [5]:
import torch.nn.functional as F  # useful stateless functions
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

### Training Loop

In [6]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    acc_max = 0
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                acc = check_accuracy(loader_val, model)
                if acc >= acc_max:
                    acc_max = acc
                print()
    print("Maximum accuracy attained: ", acc_max)

In [7]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

## Vanilla CNN; No Attention
We implement the vanilla architecture for you here. Do not modify the architecture. You will use the same architecture in the following parts. Do not modify the hyper-parameters.

In [8]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3439
Checking accuracy on validation set
Got 114 / 1000 correct (11.40)

Epoch 0, Iteration 100, loss = 1.7712
Checking accuracy on validation set
Got 405 / 1000 correct (40.50)

Epoch 0, Iteration 200, loss = 1.5565
Checking accuracy on validation set
Got 441 / 1000 correct (44.10)

Epoch 0, Iteration 300, loss = 1.5349
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Epoch 0, Iteration 400, loss = 1.6877
Checking accuracy on validation set
Got 484 / 1000 correct (48.40)

Epoch 0, Iteration 500, loss = 1.4148
Checking accuracy on validation set
Got 500 / 1000 correct (50.00)

Epoch 0, Iteration 600, loss = 1.4679
Checking accuracy on validation set
Got 532 / 1000 correct (53.20)

Epoch 0, Iteration 700, loss = 1.2731
Checking accuracy on validation set
Got 537 / 1000 correct (53.70)

Epoch 1, Iteration 0, loss = 1.0759
Checking accuracy on validation set
Got 551 / 1000 correct (55.10)

Epoch 1, Iteration 100, loss = 1.2423
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should be able to see atleast 55% accuracy

In [9]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)


Checking accuracy on test set
Got 5929 / 10000 correct (59.29)


59.29

## Part II Self-Attention

In the next section, you will implement an Attention layer which you will then use within a convnet architecture defined above for cifar 10 classification task.

A self-attention layer is formulated as following:

Input: $X$ of shape $(H\times W, C)$

Query, key, value linear transforms are $W_Q$, $W_K$, $W_V$, of shape $(C, C)$. We implement these linear transforms as 1x1 convolutional layers of the same dimensions.

$XW_Q$, $XW_K$, $XW_V$, represent the output volumes when input X is passed through the transforms.


Self-Attention is given by the formula: $Attention(X) = X + Softmax(\frac{XW_Q(XW_K)^\top}{\sqrt{C}})XW_V$

### Inline Question 1: Self-Attention is equivalent to which of the following: (5 points)
1. K-means clustering <br />
2. Non-local means <br />
3. Residual Block <br />
4. Gaussian Blurring <br />

Your Answer: <br>I choose 2. Because self-attention and non-local means both have the same process which is learning the weight distribution using relevant features, and then apply the learned weights to the features to further extract relevant knowledge.<br><br>However, the other three have different process. K-means is kind of clustering method, Residual Block use the skip connection, and the Gaussian Blurring averages the local parts.

### Here you implement the Attention module, and run it in the next section (40 points)

In [29]:
# Initialize the attention module as a nn.Module subclass
class Attention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # TODO: Implement the Key, Query and Value linear transforms as 1x1 convolutional layers
        # Hint: channel size remains constant throughout
        
        self.conv_query = nn.Conv2d(in_channels,in_channels,kernel_size=1,stride=1,padding=0, bias=False)
        self.conv_key = nn.Conv2d(in_channels,in_channels,kernel_size=1,stride=1,padding=0, bias=False)
        self.conv_value = nn.Conv2d(in_channels,in_channels,kernel_size=1,stride=1,padding=0, bias=False)
        self.softmax = nn.Softmax(dim=1)
        self.conv_mask = nn.Conv2d(in_channels,in_channels,kernel_size=1,stride=1,padding=0, bias=False)
        
    def forward(self, x):
        N, C, H, W = x.shape
        
        # TODO: Pass the input through conv_query, reshape the output volume to (N, C, H*W)
        q = self.conv_query(x).reshape(N,C,H*W)
        # TODO: Pass the input through conv_key, reshape the output volume to (N, C, H*W)
        k =  self.conv_key(x).reshape(N,C,H*W).permute(0, 2, 1).contiguous()
        # TODO: Pass the input through conv_value, reshape the output volume to (N, C, H*W)
        v =  self.conv_value(x).reshape(N,C,H*W)
        # TODO: Implement the above formula for attention using q, k, v, C
        mul = torch.matmul(q,k)
        mul = F.softmax(mul/(C**(1/2)),dim=-1)
        attention = torch.matmul(mul,v)
          
        # NOTE: The X in the formula is already added for you in the return line

        # Reshape the output to (N, C, H, W) before adding to the input volume
        attention = attention.reshape(N, C, H, W)
        return x + attention

## Single Attention Block: Early attention; After the first conv layer. (10 points)

In [30]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the first Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Linear]
model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3238
Checking accuracy on validation set
Got 98 / 1000 correct (9.80)

Epoch 0, Iteration 100, loss = 1.8953
Checking accuracy on validation set
Got 395 / 1000 correct (39.50)

Epoch 0, Iteration 200, loss = 1.5191
Checking accuracy on validation set
Got 443 / 1000 correct (44.30)

Epoch 0, Iteration 300, loss = 1.7283
Checking accuracy on validation set
Got 448 / 1000 correct (44.80)

Epoch 0, Iteration 400, loss = 1.3181
Checking accuracy on validation set
Got 486 / 1000 correct (48.60)

Epoch 0, Iteration 500, loss = 1.2694
Checking accuracy on validation set
Got 537 / 1000 correct (53.70)

Epoch 0, Iteration 600, loss = 1.1151
Checking accuracy on validation set
Got 539 / 1000 correct (53.90)

Epoch 0, Iteration 700, loss = 1.2694
Checking accuracy on validation set
Got 548 / 1000 correct (54.80)

Epoch 1, Iteration 0, loss = 0.9486
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Epoch 1, Iteration 100, loss = 1.1127
Checking accur

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should see improvement of about 2-3% over the vanilla convnet model. * Use this part to tune your Attention module and then move on to the next parts. *

In [31]:
earlyAttention = model
check_accuracy(loader_test, earlyAttention)

Checking accuracy on test set
Got 6138 / 10000 correct (61.38)


61.38

## Single Attention Block: Late attention; After the second conv layer. (10 points)

In [69]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Conv->Relu->Attention->Relu->Linear]
model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3358
Checking accuracy on validation set
Got 123 / 1000 correct (12.30)

Epoch 0, Iteration 100, loss = 1.6912
Checking accuracy on validation set
Got 455 / 1000 correct (45.50)

Epoch 0, Iteration 200, loss = 1.4778
Checking accuracy on validation set
Got 499 / 1000 correct (49.90)

Epoch 0, Iteration 300, loss = 1.3410
Checking accuracy on validation set
Got 525 / 1000 correct (52.50)

Epoch 0, Iteration 400, loss = 1.4243
Checking accuracy on validation set
Got 534 / 1000 correct (53.40)

Epoch 0, Iteration 500, loss = 1.0974
Checking accuracy on validation set
Got 553 / 1000 correct (55.30)

Epoch 0, Iteration 600, loss = 1.4289
Checking accuracy on validation set
Got 583 / 1000 correct (58.30)

Epoch 0, Iteration 700, loss = 1.2041
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Epoch 1, Iteration 0, loss = 1.0706
Checking accuracy on validation set
Got 565 / 1000 correct (56.50)

Epoch 1, Iteration 100, loss = 1.1544
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [70]:
lateAttention = model
check_accuracy(loader_test, lateAttention)

Checking accuracy on test set
Got 6007 / 10000 correct (60.07)


60.07

### Inline Question 2: Provide one example each of usage of self-attention and attention in computer vision. Explain the difference between the two. (5 points)


Your Answer:

Example for Self-Attention:In the process of lexical analysis, it can help to understand the relationship between different words.

Example for Attention: In the process of lexical analysis, it can help to the relationship between two sequences in the translation task, like the original text and the translated text

Difference:The difference between self-attention and attention is that the object of their computation has changed. the Attention mechanism occurs between the element Query of the Target and all elements in the Source. Whereas self-attention is the Attention mechanism that occurs between elements within Source or between elements within Target.

## Double Attention Blocks: After conv layers 1 and 2 (10 points)

In [71]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Attention->Relu->Linear]
model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3104
Checking accuracy on validation set
Got 133 / 1000 correct (13.30)

Epoch 0, Iteration 100, loss = 1.8292
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Epoch 0, Iteration 200, loss = 1.3537
Checking accuracy on validation set
Got 479 / 1000 correct (47.90)

Epoch 0, Iteration 300, loss = 1.1236
Checking accuracy on validation set
Got 519 / 1000 correct (51.90)

Epoch 0, Iteration 400, loss = 1.4005
Checking accuracy on validation set
Got 555 / 1000 correct (55.50)

Epoch 0, Iteration 500, loss = 1.2800
Checking accuracy on validation set
Got 563 / 1000 correct (56.30)

Epoch 0, Iteration 600, loss = 1.2889
Checking accuracy on validation set
Got 574 / 1000 correct (57.40)

Epoch 0, Iteration 700, loss = 1.2095
Checking accuracy on validation set
Got 583 / 1000 correct (58.30)

Epoch 1, Iteration 0, loss = 0.9373
Checking accuracy on validation set
Got 604 / 1000 correct (60.40)

Epoch 1, Iteration 100, loss = 1.0058
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [72]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)

Checking accuracy on test set
Got 6241 / 10000 correct (62.41)


62.41

## Resnet with Attention 

Now we will experiment with applying attention within the Resnet10 architecture that we implemented in Homework 2. Please note that for a deeper model such as Resnet we do not expect significant improvements in performance with Attention

## Vanilla Resnet, No Attention

The architecture for Resnet is given below, please train it and evaluate it on the test set.

In [63]:
import torch
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, block, layers, attention=False, img_channels=3, num_classes=100, batchnorm=False):
        super(ResNet, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.Attention = Attention(128)
        self.attention = attention

        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)
        if self.attention:
            x = self.Attention(x)
            x = self.relu(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x


        

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)
        
class block(nn.Module):

    def __init__(self, in_channels, out_channels, downsampler = None, stride = 1, batchnorm=False):
        
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampler = downsampler
        self.relu = nn.ReLU()
        self.batchnorm = batchnorm

    
    def forward(self, x):

        residual = x
        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        if self.batchnorm:
            x = self.bn2(x)
        x = self.relu(x)
        
        if self.downsampler:
            residual = self.downsampler(residual)

        return self.relu(residual + x)
    


def ResNet10(num_classes = 100, attention=False, batchnorm= False):

    return ResNet(block,[1, 1, 1, 1], attention=attention, num_classes=num_classes, batchnorm=batchnorm)

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [64]:
model = ResNet10()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 4.7208
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)

Epoch 0, Iteration 100, loss = 1.5793
Checking accuracy on validation set
Got 350 / 1000 correct (35.00)

Epoch 0, Iteration 200, loss = 1.4346
Checking accuracy on validation set
Got 469 / 1000 correct (46.90)

Epoch 0, Iteration 300, loss = 1.3628
Checking accuracy on validation set
Got 478 / 1000 correct (47.80)

Epoch 0, Iteration 400, loss = 1.2453
Checking accuracy on validation set
Got 516 / 1000 correct (51.60)

Epoch 0, Iteration 500, loss = 1.1419
Checking accuracy on validation set
Got 495 / 1000 correct (49.50)

Epoch 0, Iteration 600, loss = 1.5395
Checking accuracy on validation set
Got 575 / 1000 correct (57.50)

Epoch 0, Iteration 700, loss = 1.4707
Checking accuracy on validation set
Got 577 / 1000 correct (57.70)

Epoch 1, Iteration 0, loss = 0.9245
Checking accuracy on validation set
Got 552 / 1000 correct (55.20)

Epoch 1, Iteration 100, loss = 1.2360
Checking acc

In [65]:
vanillaResnet = model
check_accuracy(loader_test, vanillaResnet)

Checking accuracy on test set
Got 7467 / 10000 correct (74.67)


74.67

## Resnet with Attention (5 points)

In [67]:
## Resnet with Attention

learning_rate = 1e-3
model = ResNet10(num_classes = 100, attention=True, batchnorm= False)


# TODO: Use the above Attention module after the 2nd resnet block i.e. after self.layer2.


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 4.6366
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)

Epoch 0, Iteration 100, loss = 1.6535
Checking accuracy on validation set
Got 413 / 1000 correct (41.30)

Epoch 0, Iteration 200, loss = 1.4571
Checking accuracy on validation set
Got 440 / 1000 correct (44.00)

Epoch 0, Iteration 300, loss = 1.2664
Checking accuracy on validation set
Got 456 / 1000 correct (45.60)

Epoch 0, Iteration 400, loss = 1.2582
Checking accuracy on validation set
Got 514 / 1000 correct (51.40)

Epoch 0, Iteration 500, loss = 1.2246
Checking accuracy on validation set
Got 506 / 1000 correct (50.60)

Epoch 0, Iteration 600, loss = 1.2983
Checking accuracy on validation set
Got 543 / 1000 correct (54.30)

Epoch 0, Iteration 700, loss = 1.1147
Checking accuracy on validation set
Got 563 / 1000 correct (56.30)

Epoch 1, Iteration 0, loss = 1.2306
Checking accuracy on validation set
Got 581 / 1000 correct (58.10)

Epoch 1, Iteration 100, loss = 1.1982
Checking acc

## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [68]:
AttentionResnet = model
check_accuracy(loader_test, AttentionResnet)

Checking accuracy on test set
Got 7517 / 10000 correct (75.17)


75.17

## Inline Question 3: Rank the above models based on their performance on test dataset (15 points)
( You are encouraged to run each of the experiments (training) at
least 3 times to get an average estimate )

Report the test accuracies alongside the model names. For example, 1. Vanilla CNN (57.45%, 57.99%).. etc

1. Vanilla CNN without attention (57.59%，60.21%，59.38%, Average:59.06%)<br />
2. Self-Single Early Attention (60.26%,61.71%,61.38%, Average:61.12% )<br />
3. Self-Single Late Attention (58.98%,61.04%,60.07%, Average:59.93%)<br />
4. Double Attention (61.48%,63.91%, 62.47% Average:62.62%)<br />
5. Vanilla Resnet, No Attention (72.14%,73.22%,74.67%, Average:73.34%)<br />
6. Resnet with Attention (74.51%,75,12%,75.17%, Average:74.93%)<br />

So the ranking: <br>Resent with Attention> Resnet without Attention> Double Attention> Self-Single Early Attention> Self-single Late Attention>Without Attention

### Bonus Question (Ungraded): Can you give a possible explanation that supports the rankings?
Your Answer:<br>In all, algorithm with attention will show a better performance since in the paper, it says that the introduction of Self Attention will make it easier to capture long-distance interdependent features in sentences.So it shows a better result.<br> The early attention is slightly better than the late attention, the reason may be that the attention happens earlier will contribute to higher resolution.